In [2]:
import pandas as pd
import joblib
from lightgbm import LGBMClassifier

model = joblib.load('models/lgbm_model.pkl')

scaler = joblib.load('models/scaler.pkl')
encoder = joblib.load('models/encoder.pkl')

In [3]:
new_customer = {
    'gender': 'Female',
    'SeniorCitizen': 0,
    'Partner': 'Yes',
    'Dependents': 'No',
    'tenure': 5,
    'PhoneService': 'Yes',
    'MultipleLines': 'No',
    'InternetService': 'Fiber optic',
    'OnlineSecurity': 'No',
    'OnlineBackup': 'Yes',
    'DeviceProtection': 'No',
    'TechSupport': 'No',
    'StreamingTV': 'Yes',
    'StreamingMovies': 'No',
    'Contract': 'Month-to-month',
    'PaperlessBilling': 'Yes',
    'PaymentMethod': 'Electronic check',
    'MonthlyCharges': 75.35,
    'TotalCharges': 375.75
}

df_new = pd.DataFrame([new_customer])

In [4]:
def preprocess_new_data(df, scaler, encoder):
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)
    df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})
    
    binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
    for col in binary_cols:
        df[col] = df[col].map({'Yes': 1, 'No': 0})
    
    multi_cols = ['MultipleLines', 'InternetService', 'OnlineSecurity', 
                  'OnlineBackup', 'DeviceProtection', 'TechSupport',
                  'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

    encoded_multi = encoder.transform(df[multi_cols])
    encoded_cols = encoder.get_feature_names_out(multi_cols)
    encoded_df = pd.DataFrame(encoded_multi, columns=encoded_cols, index=df.index)

    df = df.drop(multi_cols, axis=1)
    df = pd.concat([df, encoded_df], axis=1)
    
    num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
    df[num_cols] = scaler.transform(df[num_cols])

    return df

In [7]:
df_processed = preprocess_new_data(df_new, scaler, encoder)
churn_prob = model.predict_proba(df_processed)[:, 1][0]

print(f"Churn Probability: {churn_prob * 100:.2f}%")
if churn_prob > 0.5:
    print("High Risk of Churn!")
else:
    print("Customer likely to stay.")

Churn Probability: 44.84%
Customer likely to stay.
